In [1]:
import os
import json
import pandas as pd
import traceback

from dotenv import load_dotenv
load_dotenv() # take environment variables from .env

True

In [2]:
KEY = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=0.5)

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import PyPDF2

In [5]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [6]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [7]:
# Defining the prompt template
mcq_generation_prompt = PromptTemplate(
    input_variables= ['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [8]:
chain = LLMChain(llm=llm, prompt=mcq_generation_prompt, output_key='quiz', verbose=True)

In [9]:
TEMPLATE_2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [10]:
mcq_eval_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [11]:
review_chain = LLMChain(llm=llm, prompt= mcq_eval_prompt, output_key="review", verbose=True)

In [12]:
# Connect chain and review_chain using SequentialChain
generate_evaluate_chain = SequentialChain(chains=[chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [13]:
file_path = r"C:\Users\SVF\McqGenerator\data.txt"

with open(file_path, 'r') as file:
    TEXT = file.read()

In [14]:
# Serialize the python dictionary into a JSON-format string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [15]:
NUMBER= 5 
SUBJECT= "machine learning"
TONE= "simple"

In [16]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.[1] Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.[2][3] Machine learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks.[4][5]

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning.[7][8]

ML is known in its application across business problems under the name 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.[1] Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.[2][3] Machine learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks.[4][5]

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning.[7][8]

ML is known in its application across business problems under the name predictive analytics. A

In [17]:
print("Total Tokens: ", cb.total_tokens)
print("Prompt Tokens: ", cb.prompt_tokens)
print("Completion Tokens: ", cb.completion_tokens)
print("Total Cost: ", cb.total_cost)

Total Tokens:  3657
Prompt Tokens:  2972
Completion Tokens:  685
Total Cost:  0.005828


In [18]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.[1] Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.[2][3] Machine learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks.[4][5]\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning.[7][8]\n\nML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational 

In [19]:
quiz = response.get('quiz')

In [20]:
quiz = json.loads(quiz)

In [21]:
data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [22]:
data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: The study of statistical algorithms in artificial intelligence | b: The study of computer vision and speech recognition | c: The study of data mining and exploratory data analysis | d: The study of mathematical optimization methods',
  'Correct': 'a'},
 {'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Arthur Samuel | b: Donald Hebb | c: Walter Pitts | d: Raytheon Company',
  'Correct': 'a'},
 {'MCQ': 'What was the early machine learning model developed by Arthur Samuel used for?',
  'Choices': 'a: Checkers | b: Sonar signals | c: Speech patterns | d: Medical diagnosis',
  'Correct': 'a'},
 {'MCQ': 'What is the relationship between machine learning and artificial intelligence?',
  'Choices': 'a: Machine learning is a subfield of AI | b: AI and machine learning are unrelated | c: AI is a subfield of machine learning | d: AI and machine learning have no relationship',
  'Correct': 'a'},
 {'MCQ': 'In which decade did

In [23]:
mcq = pd.DataFrame(data)

In [24]:
mcq.to_csv("machinelearning.csv",index=False)